In [1]:
%pip install -qU google-generativeai
%pip install -qU google-ai-generativelanguage==0.6.15
%pip install -qU langchain-google-genai
%pip install -qU langchain-community
%pip install -qU langgraph
%pip install -qU langgraph langchain-community
%pip install -qU python-dotenv

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.


...


[notice] A new release of pip is available: 24.3.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


https://react-lm.github.io/

https://smith.langchain.com/hub/langchain-ai/react-agent-template

In [1]:
import os 
import re 
import google.genai as genai 
from langgraph.graph import StateGraph, END 
from typing import TypedDict

In [3]:
from dotenv import load_dotenv
import google.generativeai as genai 

load_dotenv()

GOOGLE_API_KEY = os.getenv('GEMINI_API_KEY')
TAVILY_API_KEY = os.getenv('TAVILY_API_KEY')

genai.configure(api_key=GOOGLE_API_KEY)

model = genai.GenerativeModel('gemini-2.5-flash')
response = model.generate_content("Hola mundo")

print(response.text)

¡Hola!


In [4]:
class Agent:
    def __init__(self, system=""):
        self.system = system
        self.messages = []
        if self.system:
            self.messages.append({"role": "system", "content": system})

    def __call__(self, message):
        self.messages.append({"role": "user", "content": message})
        result = self.execute(message)
        self.messages.append({"role": "assistant", "content": result})
        return result

    def execute(self,message):
        client = genai.GenerativeModel("gemini-2.5-flash")
        completion = client.generate_content(message)
        return completion.text

In [5]:
agente = Agent(system="Eres un asistente útil y objetivo.")
print(agente("Cuál es la capital de Argentina?"))

La capital de Argentina es **Buenos Aires**.


In [6]:
PROMPT_REACT = """
Funcionas en un ciclo de Pensamiento, Acción, Pausa y Observación.
Al final del ciclo, proporcionas una Respuesta.
Usa "Pensamiento" para describir tu razonamiento.
Usa "Acción" para ejecutar herramientas - y luego retorna "PAUSA".
La "Observación" será el resultado de la acción ejecutada.
Acciones disponibles:

consultar_stock: devuelve la cantidad disponible de un artículo en el inventario (ej: "consultar_stock: teclado")

consultar_precio_producto: devuelve el precio unitario de un producto (ej: "consultar_precio_producto: mouse gamer")

Ejemplo:
Pregunta: ¿Cuántos monitores tenemos en el inventario?
Pensamiento: Debo consultar la acción consultar_stock para saber la cantidad de monitores.
Acción: consultar_stock: monitor
PAUSA

Observación: Tenemos 75 monitores en el inventario.
Respuesta: Hay 75 monitores en el inventario.
""".strip()

In [7]:
class EstadoAgente(TypedDict):
    pregunta: str
    historial: list[str]
    accion_pendiente: str
    respuesta_final: str

In [8]:
def consultar_stock(item: str) -> str:
    """
    Simula la consulta de stock de item en el inventario.
    """
    item = item.lower()
    stock = {
        "monitor": 75,
        "teclado": 120,
        "mouse de gamer": 80,
        "webcam": 40,
        "headset": 60,
        "impresora": 15
    }

    if item in stock:
        return f"Tenemos {stock[item]} {item}s en stock."
    else:
        return f"Item '{item}' no encontrado en el inventario."


def consultar_precio_producto(producto: str) -> str:
    """
    Simula la consulta del precio unitario de un producto.
    """
    producto = producto.lower()
    precios = {
        "monitor": 999.90,
        "teclado": 150.00,
        "mouse de gamer": 99.50,
        "webcam": 120.00,
        "headset": 180.00,
        "impresora": 750.00
    }

    if producto in precios:
        return f"El precio de un(a) {producto} es USD {precios[producto]:.2f}."
    else:
        return f"producto '{producto}' no hallado en la lista de precios."

In [10]:
print(consultar_stock("teclado"))
print(consultar_precio_producto("impresora"))
print(consultar_stock("monitor"))
print(consultar_stock("sillas"))

Tenemos 120 teclados en stock.
El precio de un(a) impresora es USD 750.00.
Tenemos 75 monitors en stock.
Item 'sillas' no encontrado en el inventario.


In [11]:
def run_react_agent(pregunta: str, max_iterations: int = 5) -> str:
    model = genai.GenerativeModel('gemini-2.5-flash')
    chat = model.start_chat(history=[])

    chat.send_message(PROMPT_REACT)

    current_prompt = pregunta

    for i in range(max_iterations):
        response = chat.send_message(current_prompt)
        response_text = response.text.strip()

        print(f"--- Iteración {i+1} ---")
        print(f"Modelo pensó/respondió:\n{response_text}\n")

        if response_text.startswith("Respuesta:"):
            return response_text.replace("Respuesta:", "").strip()

        match = re.search(r"Acción:\s*(\w+):\s*(.*)", response_text)

        if match:
            action_name = match.group(1).strip()
            action_arg = match.group(2).strip()

            observacion = ""
            if action_name == "consultar_stock":
                observacion = consultar_stock(action_arg)
            elif action_name == "consultar_precio_producto":
                observacion = consultar_precio_producto(action_arg)
            else:
                observacion = f"Error: Acción '{action_name}' desconocida."

            current_prompt = f"Observación: {observacion}\nRespuesta:"

            print(f"Ejecutó acción: {action_name}({action_arg})")
            print(f"Observación: {observacion}\n")

        else:
            return f"Error: El agente no logró extraer una Acción o Respuesta final tras {i+1} iteraciones. La Última respuesta fue: {response_text}"

    return "Error: Límite máximo de iteraciones alcanzado sin una respuesta final."

In [13]:
pregunta_1 = "Cuántos mouse de gamer están disponibles en el inventario?"
print(f"**Interacción 1: {pregunta_1}**")
respuesta_1 = run_react_agent(pregunta_1)
print(f"\n**RESPUESTA FINAL DEL AGENTE 1:** {respuesta_1}\n")

print("\n" + "="*50 + "\n") 

**Interacción 1: Cuántos mouse de gamer están disponibles en el inventario?**
--- Iteración 1 ---
Modelo pensó/respondió:
Pensamiento: El usuario quiere saber la cantidad disponible de "mouse de gamer". Debo usar la herramienta `consultar_stock` con ese producto.
Acción: consultar_stock: mouse de gamer
PAUSA

Ejecutó acción: consultar_stock(mouse de gamer)
Observación: Tenemos 80 mouse de gamers en stock.

--- Iteración 2 ---
Modelo pensó/respondió:
Respuesta: Tenemos 80 mouse de gamers en stock.


**RESPUESTA FINAL DEL AGENTE 1:** Tenemos 80 mouse de gamers en stock.





In [14]:
pregunta_2 = "Cuál es el precio de una impresora?"
print(f"**Interacción 2: {pregunta_2}**")
respuesta_2 = run_react_agent(pregunta_2)
print(f"\n**RESPUESTA FINAL DEL AGENTE 2:** {respuesta_2}\n")

print("\n" + "="*50 + "\n") 

**Interacción 2: Cuál es el precio de una impresora?**
--- Iteración 1 ---
Modelo pensó/respondió:
Pensamiento: Debo consultar la acción consultar_precio_producto para saber el precio de una impresora.
Acción: consultar_precio_producto: impresora
PAUSA

Ejecutó acción: consultar_precio_producto(impresora)
Observación: El precio de un(a) impresora es USD 750.00.

--- Iteración 2 ---
Modelo pensó/respondió:
Respuesta: El precio de la impresora es USD 750.00.


**RESPUESTA FINAL DEL AGENTE 2:** El precio de la impresora es USD 750.00.





In [15]:
pregunta_3 = "Tenemos sillas en inventario?"
print(f"**Interacción 3: {pregunta_3}**")
respuesta_3 = run_react_agent(pregunta_3)
print(f"\n**RESPUESTA FINAL DEL AGENTE 3:** {respuesta_3}\n")

**Interacción 3: Tenemos sillas en inventario?**
--- Iteración 1 ---
Modelo pensó/respondió:
Pensamiento: Para saber si tenemos sillas en inventario, debo consultar la cantidad disponible utilizando la herramienta `consultar_stock`.
Acción: consultar_stock: silla
PAUSA

Ejecutó acción: consultar_stock(silla)
Observación: Item 'silla' no encontrado en el inventario.

--- Iteración 2 ---
Modelo pensó/respondió:
Respuesta: No, no tenemos sillas en el inventario.


**RESPUESTA FINAL DEL AGENTE 3:** No, no tenemos sillas en el inventario.



In [16]:
pregunta_4 = "Cuál es el producto más costoso?"
print(f"**Interacción 4: {pregunta_4}**")
respuesta_4 = run_react_agent(pregunta_4)
print(f"\n**RESPUESTA FINAL DEL AGENTE 4:** {respuesta_4}\n")

**Interacción 4: Cuál es el producto más costoso?**
--- Iteración 1 ---
Modelo pensó/respondió:
Pensamiento: La pregunta busca identificar el producto más costoso. Mis herramientas disponibles son `consultar_stock` y `consultar_precio_producto`. Ambas herramientas requieren el nombre de un producto específico como entrada. No tengo una herramienta que me permita listar todos los productos disponibles o comparar los precios de todos los productos para identificar el más caro. Por lo tanto, no puedo responder directamente a esta pregunta con las acciones que tengo.

Respuesta: No puedo determinar directamente cuál es el producto más costoso porque mis herramientas solo me permiten consultar el precio de un producto específico, no comparar todos los productos existentes.


**RESPUESTA FINAL DEL AGENTE 4:** Error: El agente no logró extraer una Acción o Respuesta final tras 1 iteraciones. La Última respuesta fue: Pensamiento: La pregunta busca identificar el producto más costoso. Mis herra

In [17]:
def herramienta_encontrar_producto_mas_costoso() -> str: 
    """
    Retorna el nombre y el precio del producto más costoso en el inventario.
    Esta función no requiere argumentos adicionales.
    """
    
    precios_del_inventario = {
        "monitor": 999.90,
        "teclado": 150.00,
        "mouse de gamer": 99.50,
        "webcam": 120.00,
        "headset": 180.00,
        "impressora": 750.00
    }

    if not precios_del_inventario: 
        return "Lo sentimos, no hallamos ningún producto en la lista de precios para su comparación."

    
    nombre_producto_mas_costoso = max(precios_del_inventario, key=precios_del_inventario.get)
    valor_producto_mas_costoso = precios_del_inventario[nombre_producto_mas_costoso]

    return f"El producto más costoso es el(la) {nombre_producto_mas_costoso} con precio de USD {valor_producto_mas_costoso:.2f}." 

In [18]:
PROMPT_REACT = """
Funcionas en un ciclo de Pensamiento, Acción, Pausa y Observación.
Al final del ciclo, proporcionas una Respuesta.
Usa "Pensamiento" para describir tu razonamiento.
Usa "Acción" para ejecutar herramientas, y luego regresa "PAUSA".
La "Observación" será el resultado de la acción ejecutada.
Acciones disponibles:
  - consultar_stock: devuelve la cantidad disponible de un artículo en el inventario (ej.: "consultar_stock: teclado")
  - consultar_precio_producto: devuelve el precio unitario de un producto (ej.: "consultar_precio_producto: mouse gamer")
  - encontrar_producto_mas_costoso: devuelve el nombre y el precio del producto más costoso del inventario (no requiere argumentos)

Ejemplo:
Pregunta: ¿Cuántos monitores tenemos en stock?
Pensamiento: Debo consultar la acción consultar_stock para saber la cantidad de monitores.
Acción: consultar_stock: monitor
PAUSA

Observación: Tenemos 75 monitores en stock.
Respuesta: Hay 75 monitores en stock.

Ejemplo:
Pregunta: ¿Cuál es el producto más costoso?
Pensamiento: Necesito usar la acción encontrar_producto_mas_costoso para descubrir qué producto tiene el precio más alto.
Acción: encontrar_producto_mas_costoso
PAUSA

Observación: El producto más costoso es el monitor con un precio de R$ 999,90.
Respuesta: El producto más costoso es el monitor con un precio de R$ 999,90.
""".strip()

In [19]:
def run_react_agent(pregunta: str, max_iterations: int = 5) -> str:
    """
    Ejecuta el ciclo ReAct para una determinada pregunta usando el modelo Gemini.
    """

    model = genai.GenerativeModel('gemini-2.5-flash')
    chat = model.start_chat(history=[])
    chat.send_message(PROMPT_REACT)

    current_prompt = pregunta

    for i in range(max_iterations):
        response = chat.send_message(current_prompt)
        response_text = response.text.strip()

        print(f"\n--- Iteración {i+1} ---")
        print(f"Modelo pensó/respondió:\n{response_text}\n")

        
        response_match_final = re.search(r"Respuesta:\s*(.*)", response_text, re.DOTALL)
        if response_match_final:
            return response_match_final.group(1).strip()

        
        match = re.search(r"Acción:\s*(\w+)(?::\s*([^\n]*))?", response_text)

        if match:
            action_name = match.group(1).strip()

            action_arg = match.group(2).strip() if match.group(2) is not None else ""

            observacion_de_accion = ""

            if action_name == "consultar_stock":
                observacion_de_accion = consultar_stock(action_arg)
            elif action_name == "consultar_precio_producto":
                observacion_de_accion = consultar_precio_producto(action_arg)
            elif action_name == "encontrar_producto_mas_costoso": 
                observacion_de_accion = herramienta_encontrar_producto_mas_costoso()
            else:
                observacion_de_accion = f"Error: Acción '{action_name}' desconocida. Verifica el prompt o la implementación de la herramienta."

            # Simplificamos el prompt de Observación
            current_prompt = f"Observación: {observacion_de_accion}"

            print(f"Ejecutó acción: {action_name} con argumento '{action_arg}'")
            print(f"Observación: {observacion_de_accion}\n")

        else:
            
            return f"Error: El agente no logró extraer una Acción o Respuesta final tras {i+1} iteraciones. La Última respuesta fue: {response_text}"

    return "Error: Límite máximo de iteraciones alcanzado sin una respuesta final."

In [20]:
print("--- Iniciando a interacctuar con el Agente ReAct ---")

# Interacción 1: Consultar Estoque
pregunta_1 = "Cuántos teclados tenemos en stock?"
print(f"\n**Interacción 1: {pregunta_1}**")
respuesta_1 = run_react_agent(pregunta_1)
print(f"\n**RESPUESTA FINAL DEL AGENTE 1:** {respuesta_1}\n")

print("\n" + "="*80 + "\n")

# Interacción 2: Consultar Preço
pregunta_2 = "Cuál es el precio de un headset?"
print(f"\n**Interacción 2: {pregunta_2}**")
respuesta_2 = run_react_agent(pregunta_2)
print(f"\n**RESPUESTA FINAL DEL AGENTE 2:** {respuesta_2}\n")

print("\n" + "="*80 + "\n")

# Interacción 3: Item não encontrado no estoque
pregunta_3 = "Tenemos sillas en stock?"
print(f"\n**Interacción 3: {pregunta_3}**")
respuesta_3 = run_react_agent(pregunta_3)
print(f"\n**RESPUESTA FINAL DEL AGENTE 3:** {respuesta_3}\n")

print("\n" + "="*80 + "\n")

# Interacción 4: Encontrar o Produto Mais Caro (A nova funcionalidade!)
pregunta_4 = "Cuál es el producto más costoso?"
print(f"\n**Interacción 4: {pregunta_4}**")
respuesta_4 = run_react_agent(pregunta_4)
print(f"\n**RESPUESTA FINAL DEL AGENTE 4:** {respuesta_4}\n")

print("\n--- Fin de las Interacciones ---")

--- Iniciando a interacctuar con el Agente ReAct ---

**Interacción 1: Cuántos teclados tenemos en stock?**

--- Iteración 1 ---
Modelo pensó/respondió:
Pensamiento: Debo consultar la acción consultar_stock para saber la cantidad de teclados.
Acción: consultar_stock: teclado
PAUSA

Ejecutó acción: consultar_stock con argumento 'teclado'
Observación: Tenemos 120 teclados en stock.


--- Iteración 2 ---
Modelo pensó/respondió:
Respuesta: Hay 120 teclados en stock.


**RESPUESTA FINAL DEL AGENTE 1:** Hay 120 teclados en stock.




**Interacción 2: Cuál es el precio de un headset?**

--- Iteración 1 ---
Modelo pensó/respondió:
Pensamiento: Debo usar la acción `consultar_precio_producto` para saber el precio de un headset.
Acción: consultar_precio_producto: headset
PAUSA

Ejecutó acción: consultar_precio_producto con argumento 'headset'
Observación: El precio de un(a) headset es USD 180.00.


--- Iteración 2 ---
Modelo pensó/respondió:
Respuesta: El precio de un headset es USD 180.00.


**R

In [21]:
def run_react_agent_with_history(pregunta: str, max_iterations: int = 5) -> tuple[str, list]: 
    model = genai.GenerativeModel('gemini-2.5-flash') 

    chat = model.start_chat(history=[])
    chat.send_message(PROMPT_REACT) 

    current_prompt = pregunta

    for i in range(max_iterations):
        response = chat.send_message(current_prompt)
        response_text = response.text.strip()

        print(f"\n--- Iteración {i+1} ---")
        print(f"Modelo pensó/respondió:\n{response_text}\n")

        response_match_final = re.search(r"Respuesta:\s*(.*)", response_text, re.DOTALL)
        if response_match_final:
            return response_match_final.group(1).strip(), chat.history

        match = re.search(r"Acción:\s*(\w+)(?::\s*([^\n]*))?", response_text)

        if match:
            action_name = match.group(1).strip()
            action_arg = match.group(2).strip() if match.group(2) is not None else ""

            observacion_de_accion = ""

            if action_name == "consultar_stock":
                observacion_de_accion = consultar_stock(action_arg)
            elif action_name == "consultar_precio_producto":
                observacion_de_accion = consultar_precio_producto(action_arg)
            elif action_name == "encontrar_producto_mas_costoso":
                observacion_de_accion = herramienta_encontrar_producto_mas_costoso()
            else:
                observacion_de_accion = f"Error: Acción '{action_name}' desconocida. Verifica el prompt o la implementación de la herramienta."

            current_prompt = f"Observación: {observacion_de_accion}"

            print(f"Ejecutó la acción: {action_name} con argumento '{action_arg}'")
            print(f"Observación: {observacion_de_accion}\n")

        else:
            return f"Error: El agente no logró extraer una Acción o Respuesta final tras {i+1} iteraciones. La Última respuesta fue: {response_text}", chat.history

    return "Error: Límite máximo de iteraciones alcanzado sin una respuesta final.", chat.history

In [22]:
pregunta_ejemplo = "Cuántos teclados tenemos en stock?"
respuesta_ejemplo, historial_completo = run_react_agent_with_history(pregunta_ejemplo)

print(f"**RESPUESTA FINAL DEL AGENTE:** {respuesta_ejemplo}\n")

print("\n--- Historial Completo de la Interacción ---")
for i, message in enumerate(historial_completo):
    print(f"--- Mensaje {i+1} (rol: {message.role}) ---")

    for part in message.parts:
        if hasattr(part, 'text'):
            print(part.text)
        else:
            print(part)
    print("-" * 20)
print("\n--- Fin del historial ---\n")


--- Iteración 1 ---
Modelo pensó/respondió:
Pensamiento: Para saber cuántos teclados hay en stock, debo usar la acción `consultar_stock` con el argumento "teclado".
Acción: consultar_stock: teclado
PAUSA

Ejecutó la acción: consultar_stock con argumento 'teclado'
Observación: Tenemos 120 teclados en stock.


--- Iteración 2 ---
Modelo pensó/respondió:
Respuesta: Hay 120 teclados en stock.

**RESPUESTA FINAL DEL AGENTE:** Hay 120 teclados en stock.


--- Historial Completo de la Interacción ---
--- Mensaje 1 (rol: user) ---
Funcionas en un ciclo de Pensamiento, Acción, Pausa y Observación.
Al final del ciclo, proporcionas una Respuesta.
Usa "Pensamiento" para describir tu razonamiento.
Usa "Acción" para ejecutar herramientas, y luego regresa "PAUSA".
La "Observación" será el resultado de la acción ejecutada.
Acciones disponibles:
  - consultar_stock: devuelve la cantidad disponible de un artículo en el inventario (ej.: "consultar_stock: teclado")
  - consultar_precio_producto: devuelve

In [23]:
def herramienta_calcular_valor_total_lista(lista_items: str) -> str:
    """
    Calcula el valor total de una lista de items de compra.
    Recibe una string con items separados por coma (ex: "teclado, mouse de gamer, monitor").
    """
    precios_del_inventario = { 
        "monitor": 999.90,
        "teclado": 150.00,
        "mouse de gamer": 99.50,
        "webcam": 120.00,
        "headset": 180.00,
        "impresora": 750.00
    }

    items_procesados = [item.strip().lower() for item in lista_items.split(',')]

    valor_total = 0.0
    items_no_encontrados = []


    for item in items_procesados: 
        if item in precios_del_inventario:
            valor_total += precios_del_inventario[item]
        else:
            items_no_encontrados.append(item)

    respuesta = f"El valor total de los items encontrados es USD {valor_total:.2f}."
    if items_no_encontrados:
        respuesta += f"Los siguientes items no fueron encontrados y tampoco incluídos en el cálculo: {', '.join(items_no_encontrados)}."

    return respuesta


In [24]:
print("Testando herramienta_calcular_valor_total_lista:")

lista_1 = "teclado, mouse de gamer, monitor"
resultado_1 = herramienta_calcular_valor_total_lista(lista_1)
print(f"Lista: '{lista_1}'\nResultado: {resultado_1}\n")

Testando herramienta_calcular_valor_total_lista:
Lista: 'teclado, mouse de gamer, monitor'
Resultado: El valor total de los items encontrados es USD 1249.40.



In [25]:
lista_2 = "headset, silla"
resultado_2 = herramienta_calcular_valor_total_lista(lista_2)
print(f"Lista: '{lista_2}'\nResultado: {resultado_2}\n")

Lista: 'headset, silla'
Resultado: El valor total de los items encontrados es USD 180.00.Los siguientes items no fueron encontrados y tampoco incluídos en el cálculo: silla.



In [26]:
lista_3 = "mesa, vaso"
resultado_3 = herramienta_calcular_valor_total_lista(lista_3)
print(f"Lista: '{lista_3}'\nResultado: {resultado_3}\n")


Lista: 'mesa, vaso'
Resultado: El valor total de los items encontrados es USD 0.00.Los siguientes items no fueron encontrados y tampoco incluídos en el cálculo: mesa, vaso.



In [27]:
def run_react_agent(pregunta: str, max_iterations: int = 5) -> str:
    model = genai.GenerativeModel('gemini-2.5-flash')

    chat = model.start_chat(history=[])
    chat.send_message(PROMPT_REACT)

    current_prompt = pregunta

    for i in range(max_iterations):
        response = chat.send_message(current_prompt)
        response_text = response.text.strip()

        print(f"\n--- Iteración {i+1} ---")
        print(f"Modelo pensó/respondió:\n{response_text}\n")

        response_match_final = re.search(r"Respuesta:\s*(.*)", response_text, re.DOTALL)
        if response_match_final:
            return response_match_final.group(1).strip()


        match = re.search(r"Acción:\s*(\w+)(?::\s*([^\n]*))?", response_text)

        if match:
            action_name = match.group(1).strip()
            action_arg = match.group(2).strip() if match.group(2) is not None else ""

            observacion_de_accion = ""

            if action_name == "consultar_stock":
                observacion_de_accion = consultar_stock(action_arg)
            elif action_name == "consultar_precio_producto":
                observacion_de_accion = consultar_precio_producto(action_arg)
            elif action_name == "encontrar_produto_mais_costoso":
                observacion_de_accion = herramienta_encontrar_producto_mas_costoso()
            elif action_name == "calcular_valor_total_lista":
                observacion_de_accion = herramienta_calcular_valor_total_lista(action_arg)
            else:
                observacion_de_accion = f"Error: Acción '{action_name}' desconocida. Verifica el prompt o la implementación de la herramienta."

            current_prompt = f"Observación: {observacion_de_accion}"

            print(f"Ejecutó acción: {action_name} con argumento '{action_arg}'")
            print(f"Observación: {observacion_de_accion}\n")

        else:
            return f"Error: El agente no logró extraer una Acción o Respuesta final tras {i+1} iteraciones. La Última respuesta fue: {response_text}"

    return "Error: Límite máximo de iteraciones alcanzado sin una respuesta final."

In [28]:
print("--- Iniciando a interacctuar con el Agente ReAct ---")

# Interacción 1: Consultar Stock
pregunta_1 = "Cuántos teclados tenemos en stock?"
print(f"\n**Interacción 1: {pregunta_1}**")
respuesta_1 = run_react_agent(pregunta_1)
print(f"\n**RESPUESTA FINAL DEL AGENTE 1:** {respuesta_1}\n")

print("\n" + "="*80 + "\n")

# Interacción 2: Consultar Precio
pregunta_2 = "Cuál es el precio de un headset?"
print(f"\n**Interacción 2: {pregunta_2}**")
respuesta_2 = run_react_agent(pregunta_2)
print(f"\n**RESPUESTA FINAL DEL AGENTE 2:** {respuesta_2}\n")

print("\n" + "="*80 + "\n")

# Interacción 3: Item no encontrado en stock
pregunta_3 = "Tenemos sillas en stock?"
print(f"\n**Interacción 3: {pregunta_3}**")
respuesta_3 = run_react_agent(pregunta_3)
print(f"\n**RESPUESTA FINAL DEL AGENTE 3:** {respuesta_3}\n")

print("\n" + "="*80 + "\n")

# Interacción 4: Encontrar el Produto Más Costoso
pregunta_4 = "Cuál es el producto más costoso?"
print(f"\n**Interacción 4: {pregunta_4}**")
respuesta_4 = run_react_agent(pregunta_4)
print(f"\n**RESPUESTA FINAL DEL AGENTE 4:** {respuesta_4}\n")

print("\n" + "="*80 + "\n")

# Interacción 5: Calcular el Valor Total de la Lista (NUEVA FUNCIONALIDAD)
pregunta_5 = "Cuál es el valor de un teclado, una impresora y una webcam?"
print(f"\n**Interacción 5: {pregunta_5}**")
respuesta_5 = run_react_agent(pregunta_5)
print(f"\n**RESPUESTA FINAL DEL AGENTE 5:** {respuesta_5}\n")

print("\n--- Fin de las Interacciones ---")

--- Iniciando a interacctuar con el Agente ReAct ---

**Interacción 1: Cuántos teclados tenemos en stock?**

--- Iteración 1 ---
Modelo pensó/respondió:
Pensamiento: Debo usar la acción `consultar_stock` para saber la cantidad de teclados.
Acción: consultar_stock: teclado
PAUSA

Ejecutó acción: consultar_stock con argumento 'teclado'
Observación: Tenemos 120 teclados en stock.


--- Iteración 2 ---
Modelo pensó/respondió:
Respuesta: Hay 120 teclados en stock.


**RESPUESTA FINAL DEL AGENTE 1:** Hay 120 teclados en stock.




**Interacción 2: Cuál es el precio de un headset?**

--- Iteración 1 ---
Modelo pensó/respondió:
Pensamiento: Debo usar la acción `consultar_precio_producto` para encontrar el precio de un headset.
Acción: consultar_precio_producto: headset
PAUSA

Ejecutó acción: consultar_precio_producto con argumento 'headset'
Observación: El precio de un(a) headset es USD 180.00.


--- Iteración 2 ---
Modelo pensó/respondió:
Respuesta: El precio de un headset es USD 180.00.


**

In [29]:
PROMPT_REACT = """
Funcionas en un ciclo de Pensamiento, Acción, Pausa y Observación.
Al final del ciclo, proporcionas una Respuesta.
Usa "Pensamiento" para describir tu razonamiento.
Usa "Acción" para ejecutar herramientas, y luego retorna "PAUSA".
La "Observación" será el resultado de la acción ejecutada.
Acciones disponibles:

consultar_stock: devuelve la cantidad disponible de un artículo en el inventario (ej: "consultar_stock: teclado")

consultar_precio_producto: devuelve el precio unitario de un producto (ej: "consultar_precio_producto: mouse de gamer")

encontrar_producto_mas_costoso: devuelve el nombre y el precio del producto más costoso del inventario (no requiere argumentos)

calcular_valor_total_lista: calcula el valor total de una lista de artículos de compra. Recibe una cadena con artículos separados por comas (ej: "teclado, mouse de gamer, monitor")

Ejemplo:
Pregunta: ¿Cuántos monitores tenemos en stock?
Pensamiento: Debo consultar la acción consultar_stock para saber la cantidad de monitores.
Acción: consultar_stock: monitor
PAUSA

Observación: Tenemos 75 monitores en stock.
Respuesta: Hay 75 monitores en stock.

Ejemplo:
Pregunta: ¿Cuál es el producto más costoso?
Pensamiento: Necesito usar la acción encontrar_producto_mas_costoso para descubrir qué producto tiene el mayor precio.
Acción: encontrar_producto_mas_costoso
PAUSA

Observación: El producto más costoso es el monitor con un precio de R$ 999.90.
Respuesta: El producto más costoso es el monitor con un precio de R$ 999.90.

Ejemplo:
Pregunta: ¿Cuánto cuesta un teclado y un mouse de gamer?
Pensamiento: El usuario quiere saber el valor total de varios artículos. Debo usar la acción calcular_valor_total_lista con los artículos "teclado, mouse de gamer".
Acción: calcular_valor_total_lista: teclado, mouse de gamer
PAUSA

Observación: El valor total de los artículos encontrados es R$ 249.50.
Respuesta: El valor total del teclado y del mouse de gamer es R$ 249.50.
""".strip()

In [30]:
print("--- Iniciando a interacctuar con el Agente ReAct ---")

# Interacción 1: Consultar Stock
pregunta_1 = "Cuántos teclados tenemos en stock?"
print(f"\n**Interacción 1: {pregunta_1}**")
respuesta_1 = run_react_agent(pregunta_1)
print(f"\n**RESPUESTA FINAL DEL AGENTE 1:** {respuesta_1}\n")

print("\n" + "="*80 + "\n")

# Interacción 2: Consultar Precio
pregunta_2 = "Cuál es el precio de un headset?"
print(f"\n**Interacción 2: {pregunta_2}**")
respuesta_2 = run_react_agent(pregunta_2)
print(f"\n**RESPUESTA FINAL DEL AGENTE 2:** {respuesta_2}\n")

print("\n" + "="*80 + "\n")

# Interacción 3: Item no encontrado en stock
pregunta_3 = "Tenemos sillas en stock?"
print(f"\n**Interacción 3: {pregunta_3}**")
respuesta_3 = run_react_agent(pregunta_3)
print(f"\n**RESPUESTA FINAL DEL AGENTE 3:** {respuesta_3}\n")

print("\n" + "="*80 + "\n")

# Interacción 4: Encontrar el Produto Más Costoso
pregunta_4 = "Cuál es el producto más costoso?"
print(f"\n**Interacción 4: {pregunta_4}**")
respuesta_4 = run_react_agent(pregunta_4)
print(f"\n**RESPUESTA FINAL DEL AGENTE 4:** {respuesta_4}\n")

print("\n" + "="*80 + "\n")

# Interacción 5: Calcular el Valor Total de la Lista (NUEVA FUNCIONALIDAD)
pregunta_5 = "Cuál es el valor de un teclado, una impresora y una webcam?"
print(f"\n**Interacción 5: {pregunta_5}**")
respuesta_5 = run_react_agent(pregunta_5)
print(f"\n**RESPUESTA FINAL DEL AGENTE 5:** {respuesta_5}\n")

print("\n--- Fin de las Interacciones ---")

--- Iniciando a interacctuar con el Agente ReAct ---

**Interacción 1: Cuántos teclados tenemos en stock?**

--- Iteración 1 ---
Modelo pensó/respondió:
Pensamiento: Debo consultar la acción consultar_stock para saber la cantidad de teclados.
Acción: consultar_stock: teclado
PAUSA

Ejecutó acción: consultar_stock con argumento 'teclado'
Observación: Tenemos 120 teclados en stock.


--- Iteración 2 ---
Modelo pensó/respondió:
Respuesta: Hay 120 teclados en stock.


**RESPUESTA FINAL DEL AGENTE 1:** Hay 120 teclados en stock.




**Interacción 2: Cuál es el precio de un headset?**

--- Iteración 1 ---
Modelo pensó/respondió:
Pensamiento: El usuario quiere saber el precio de un "headset". Debo usar la acción `consultar_precio_producto` para obtener esta información.
Acción: consultar_precio_producto: headset
PAUSA

Ejecutó acción: consultar_precio_producto con argumento 'headset'
Observación: El precio de un(a) headset es USD 180.00.


--- Iteración 2 ---
Modelo pensó/respondió:
Respuesta

In [31]:
def iniciar_conversacion_con_agente():
    print("--- Agente de Inventario Interactivo ---")
    print("Realiza tu pregunta sobre el inventario, o digita 'salir' para cerrar la sesión.")
    print("-" * 50)

    while True:
        pregunta_usuario = input("\nUsted: ")

        if pregunta_usuario.lower().strip() == 'salir':
            print("Atención finalizada. Hasta pronto!")
            break

        print("\nAgente: Procesando...")
        try:

            respuesta_agente = run_react_agent(pregunta_usuario)
            print(f"\nAgente: {respuesta_agente}")
        except Exception as e:

            print(f"\nAgente: Ocurrió un error al procesar su pregunta: {e}")
            print("Por favor, intenta nuevamente, o digita 'salir'.")

In [32]:
iniciar_conversacion_con_agente()

--- Agente de Inventario Interactivo ---
Realiza tu pregunta sobre el inventario, o digita 'salir' para cerrar la sesión.
--------------------------------------------------

Agente: Procesando...

--- Iteración 1 ---
Modelo pensó/respondió:
Pensamiento: El usuario quiere saber el valor total de varios artículos. Debo usar la acción `calcular_valor_total_lista` y listar los artículos "silla, silla, teclado" para representar las dos sillas y el teclado.
Acción: calcular_valor_total_lista: silla, silla, teclado
PAUSA

Ejecutó acción: calcular_valor_total_lista con argumento 'silla, silla, teclado'
Observación: El valor total de los items encontrados es USD 150.00.Los siguientes items no fueron encontrados y tampoco incluídos en el cálculo: silla, silla.


--- Iteración 2 ---
Modelo pensó/respondió:
Respuesta: El valor total de los artículos encontrados es USD 150.00. Las sillas no fueron encontradas y no se incluyeron en el cálculo.


Agente: El valor total de los artículos encontrados es